# Convolutional Neural Networks in hls4ml

In this notebook you will learn how to train a pruned and quantized convolutional neural network (CNN) and deploy it using hls4ml. For this exercise, we will use the Street View House Numbers (SVHN) Dataset (http://ufldl.stanford.edu/housenumbers/).

The SVHN dataset consists of real-world images of house numbers extracted from Google Street View images. The format is similar to that of the MNIST dataset, but is a much more challenging real-world problem, as illustrated by the examples shown below.

All the images are in RGB format and have been cropped to 32x32 pixels. 
Unlike MNIST, more than one digit can be present in the same image and in these cases, the center digit is used to assign a label to the image.
Each image can belong to one of 10 classes, corresponding to digits 0 through 9.

![alt text](images/test.png "SVHN examples from the test dataset")

The SVHN dataset consists of 73,257 images for training (and 531,131 extra samples that are easier to classify and can be used as additional training data) and 26,032 images for testing.

### Start with the neccessary imports

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
AUTO = tf.data.experimental.AUTOTUNE
import tensorflow_datasets as tfds

### Fetch the SVHN dataset using Tensorflow Dataset

In this part we will fetch the trainining, validation and test dataset using Tensorflow Datasets (https://www.tensorflow.org/datasets). We will not use the 'extra' training in order to save time, but you could fetch it by adding `split='train[:90%]+extra'`. We will use the first 90% of the training data for training and the last 10% for validation.

In [19]:
ds_train, info = tfds.load('svhn_cropped', split='train[:90%]', with_info=True, data_dir="/eos/home-t/thaarres/tensorflow_datasets/", as_supervised=True)
ds_test        = tfds.load('svhn_cropped', split='test', shuffle_files=True, data_dir="/eos/home-t/thaarres/tensorflow_datasets/", as_supervised=True)
ds_val         = tfds.load('svhn_cropped', split='train[-10%:]', shuffle_files=True, data_dir="/eos/home-t/thaarres/tensorflow_datasets/", as_supervised=True)

assert isinstance(ds_train, tf.data.Dataset)
print(ds_train)

<PrefetchDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>


We'll use TensorFlow Dataset to prepare our datasets

In [22]:
def preprocess(image, label,nclasses=10):
  image = tf.cast(image, tf.float32) / 255.
  label = tf.one_hot(tf.squeeze(label), nclasses)
  return image, label

batch_size = 1024
epochs = 30

train = ds_train.map(preprocess) #Get dataset as image and one-hot encoded labels, divided by max RGB    
train = train.repeat()
train = train.batch(batch_size) # Prepare batches
train = train.prefetch(AUTO) # Allows later elements to be prepared while the current element is being processed

val = ds_val.map(preprocess)    
val = val.batch(batch_size)
val = val.prefetch(AUTO)
  
test  = ds_test.map(preprocess)
test  = test.batch(batch_size)
test  = test.prefetch(AUTO)
  
train_size = int(info.splits['train'].num_examples)

print('Training on {} samples'.format(train_size))

Training on 73257 samples


### Defining the model

We then need to define a model. We want the lowest latency model possible and therefore limit the size per layer to be below 4096. The reason for this

In [25]:
steps_per_epoch      = int(train_size*0.9)  // batch_size #90% train, 10% validation in 10-fold xval
eval_steps_per_epoch = int(train_size*0.1) //  batch_size

LOSS        = tf.keras.losses.CategoricalCrossentropy()
OPTIMIZER   = tf.keras.optimizers.Adam(learning_rate=3E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True) 
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])
model.summary()
  
    